In [1]:
import sys
sys.path.append("C:/Users/E1005279/OneDrive - Sanlam Life Insurance Limited/MWL/MWLrepo/MiWayLife2/Data_Analysis_Package")
from data_preparation import DataPreparation, Analysis, multi_data_ops
import data_operations as do

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime

import scipy.stats as stats
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import STL
import math
import matplotlib.gridspec as gridspec


import data_transforations as data_tr

from sklearn import tree

# parameters

In [2]:
current_date='2025-08-01'

# loading data

In [3]:
path = "C:/Users/E1005279/OneDrive - Sanlam Life Insurance Limited/MWL/Lapse Rate Projections/Data/"

calls_data = pd.DataFrame(pd.read_csv(path+"call_reason.csv"))
sales_data = pd.DataFrame(pd.read_csv(path+"sales.csv"))
policy_data = pd.DataFrame(pd.read_csv(path+"policy_data.csv"))
payment_data = pd.concat([pd.DataFrame(pd.read_csv(path+"Payment_data_1.csv")),pd.DataFrame(pd.read_csv(path+"Payment_data_2.csv"))])
lapse_data = pd.concat([pd.DataFrame(pd.read_csv(path+"lapse_1.csv")),pd.DataFrame(pd.read_csv(path+"lapse_2.csv"))])

## Filtering data to relevant policies

Relevant policies are policies that have been incepted. The rpt_datamodel_lapse has incepted policies only, hence filtering on policies within that dataset

In [4]:
pol_names = list(lapse_data['policy_name'])
sales_data = sales_data[sales_data['policy_name'].isin(pol_names)]
pol_ids= list(sales_data['policy_id'])
calls_data = calls_data[calls_data['policy_name'].isin(pol_names)]
payment_data = payment_data[payment_data['policy_id'].isin(pol_ids)]
policy_data = policy_data[policy_data['policy_name'].isin(pol_names)]

#Make date column of data type datetime
sales_data=do.dtype_datetime(sales_data)
calls_data=do.dtype_datetime(calls_data)
payment_data=do.dtype_datetime(payment_data)
policy_data=do.dtype_datetime(policy_data)
lapse_data=do.dtype_datetime(lapse_data)


## Aggregating call, payment, sales and lapse data then merging

In [5]:
pre_inception_calls_data = data_tr.pre_sale_calls(calls_data,policy_data)
post_sale_calls_data = data_tr.post_sale_calls(calls_data,policy_data)
payment_hist_data = data_tr.payments_history(payment_data,current_date=current_date)
lapse_data = data_tr.lapses(lapse_data)
sales_data = sales_data.drop_duplicates(subset=['policy_id'], keep='last')

c:\Users\E1005279\OneDrive - Sanlam Life Insurance Limited\MWL\MWLrepo\LapseRate_modeling\data_transforations.py:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calls.df['res_code']= calls.df['resolution_code'].apply(res_code_categories)
c:\Users\E1005279\OneDrive - Sanlam Life Insurance Limited\MWL\MWLrepo\LapseRate_modeling\data_transforations.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calls.df['res_code']= calls.df['resolution_code'].apply(res_code_categories_postsale)
c:\Users\E1005279\OneD

In [6]:
inception_data = data_tr.at_inception_data_merge(pre_inception_calls_data, sales_data,policy_data,lapse_data)

In [6]:

near_ftr_lapse_data = data_tr.post_sale_data_merge(post_sale_calls_data, sales_data,policy_data,lapse_data,payment_hist_data)

## Some cleaning and filtering

In [7]:

inception_data = data_tr.at_inception_data_clean_filter(inception_data)

In [7]:
near_ftr_lapse_data = data_tr.near_ftr_lapse_data_clean_filter(near_ftr_lapse_data)

## Adding economic indicators

In [8]:
path = "C:/Users/E1005279/OneDrive - Sanlam Life Insurance Limited/MWL/Lapse Rate Projections/Data/"
unemployment = pd.DataFrame(pd.read_csv(path+"unemployment rates.csv"))
unemployment= unemployment[['year','ave']]
inflation = pd.DataFrame(pd.read_csv(path+"inflation.csv"))

In [9]:

inception_data['sale_date_year'] = pd.to_datetime(inception_data['sale_date']).dt.year
inception_data = data_tr.adding_eco_ind(inflation, unemployment,inception_data,'sale_date_year')

In [9]:
near_ftr_lapse_data['cover_start_date_year' ] = pd.to_datetime(near_ftr_lapse_data['cover_start_date' ]).dt.year
near_ftr_lapse_data = data_tr.adding_eco_ind(inflation, unemployment,near_ftr_lapse_data,'cover_start_date_year')

## Target creation

In [10]:
inception_data = data_tr.inception_targets(inception_data)


In [10]:
near_ftr_lapse_data = data_tr.near_ftr_targets(near_ftr_lapse_data)

In [11]:
near_ftr_lapse_data.head()

,policy_id,policy_name,age,benefits_count,education,gender,income,smoker_status,sold_socio_economic_class,securitygroup_id,...,res_code_QA_sum,res_code_claims_sum,res_code_other_sum,res_code_payment_sum,res_code_sale_sum,res_code_update_sum,cover_start_date_year,inflation,unemployment rate,payment lapse
0,100003c9-2094-3107-e604-65d8e6b4ec03,MWL008050072,51,1.0,Matric,Male,40000.0,S,1.0,MWL,...,0.0,0.0,0.0,0.0,0.0,0.0,2021,4.5,34.300,NaN
1,10000cc8-8f08-5e2c-b6ad-661d48606710,ML24-0199554,23,1.0,Matric + 3 year Diploma,Female,20000.0,N,2.0,MWL,...,0.0,0.0,0.0,0.0,0.0,0.0,2024,4.4,32.600,NaN
2,100307d2-6857-7f50-833c-66b391c37879,ML24-0254853,63,1.0,Matric + 3 year Diploma,Female,45000.0,E,1.0,MWL,...,0.0,0.0,0.0,3.0,1.0,0.0,2024,4.4,32.600,NaN
3,10048ce0-253d-aeff-4f59-64e76756bddf,ML23-0067459,41,1.0,Matric + 3 year Diploma,Male,70000.0,S,1.0,MWL,...,0.0,0.0,0.0,0.0,0.0,0.0,2023,6.0,32.375,NaN
4,10061226-ed6a-b733-47f0-65fb970086cd,MWL007912004,36,5.0,4 year Degree,Female,40000.0,N,1.0,MWL,...,0.0,0.0,0.0,0.0,0.0,0.0,2021,4.5,34.300,NaN


In [12]:
near_ftr_lapse_data.to_csv(path+'NearFuture/near_ftr_model_data.csv')

## grouping continous variables

In [65]:
#gropuping function using decision trees
from sklearn.tree import DecisionTreeClassifier, export_text
from sklearn.model_selection import train_test_split


def group_by_decision_tree(model, column_data, column_name):
    """
    Groups a pandas Series into categorical bands based on the leaf nodes of a
    fitted Decision Tree Classifier model.

    This function first traverses the decision tree to determine the value ranges
    for each leaf node. It then applies the model to the input column data to get
    the leaf node index for each data point and uses the collected ranges to
    assign a categorical band.

    Args:
        model (sklearn.tree.DecisionTreeClassifier): A fitted decision tree model.
        column_data (pd.Series): The pandas Series to be grouped.
        column_name (str): The name of the column in the original DataFrame
                           that the decision tree was trained on.

    Returns:
        pd.Series: A new pandas Series containing the categorical bands.
    """
    # Ensure the model is a decision tree
    if not isinstance(model, DecisionTreeClassifier):
        raise TypeError("The provided model must be an instance of DecisionTreeClassifier.")
    
    tree = model.tree_
    
    # Check if the tree is empty or not fitted
    if tree.node_count == 0:
        raise ValueError("The decision tree model does not seem to be fitted.")

    # A helper dictionary to store the min/max range for each leaf node
    leaf_ranges = {}
    
    def traverse_tree(node_index, lower_bound=float('-inf'), upper_bound=float('inf')):
        """
        Recursively traverses the tree to find the value range for each leaf node.
        """
        # Base case: if it's a leaf node
        if tree.children_left[node_index] == tree.children_right[node_index]:
            leaf_ranges[node_index] = (lower_bound, upper_bound)
            return

        threshold = tree.threshold[node_index]
        left_child = tree.children_left[node_index]
        right_child = tree.children_right[node_index]

        # Recurse down the left child (less than or equal to threshold)
        traverse_tree(left_child, lower_bound, min(upper_bound, threshold))

        # Recurse down the right child (greater than threshold)
        traverse_tree(right_child, max(lower_bound, threshold), upper_bound)

    # Start the traversal from the root node (index 0)
    traverse_tree(0)

    # Use the model's apply method to get the leaf node for each data point
    # We need to reshape the data to a 2D array for the model
    leaf_indices = model.apply(column_data.values.reshape(-1, 1))

    # Create the new column of grouped bands
    grouped_bands = []
    for index in leaf_indices:
        lower, upper = leaf_ranges[index]

        # Format the string for the band
        if lower == float('-inf'):
            band = f"< {upper:.2f}"
        elif upper == float('inf'):
            band = f"> {lower:.2f}"
        else:
            band = f"{lower:.2f} - {upper:.2f}"
        
        grouped_bands.append(band)

    return pd.Series(grouped_bands, index=column_data.index, name=f'{column_name}_bands')

In [66]:
inception_data.drop(columns=['orginal individual_income', 'policy_id', 'sale_date_year', 'policy_type',
       'policy_status','occupation','campaign_name','hiv_group',
       'current_policy_status', 'cancellation_effective_date',
       'cancellation_reason', 'lapse_type','cover_start_date', 'end_date', 'pol_duration'],inplace=True)
cont_var_list=['age','income',  'last_benefit_amount', 'cover_amount_full', 'total_funeral_premium',  'original_premium']
target='lapse_flag'
df=inception_data.copy()
for variable in cont_var_list:
    X = df[variable].values.reshape(-1, 1)
    Y = df[target]
    clf=tree.DecisionTreeClassifier(min_samples_leaf=math.floor(0.15*inception_data.shape[0]))
    clf=clf.fit(X,Y)
    df[variable+'_band'] = group_by_decision_tree(clf, df[variable], df)
inception_data=df

In [67]:
inception_data.to_csv(path+'AtIncept-2025/at_incpt_data.csv')